# How to upload multiple files for an object classification project using Custom Vision from Microsoft

This sample code will let you upload multiple image files, and their related tags, in one batch. It can be useful when you have to use multiple files for your object classification project.

Let's start by creating the project. Remember to replace the keys with your own keys! You might also want to change the project name.

In [1]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry

# Replace with a valid key
training_key = "<INSERT YOUR TRAINING KEY>"
prediction_key = "<INSERT YOUR PREDICTION KEY>"

trainer = training_api.TrainingApi(training_key)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Object Classification Multiple Files")

Creating project...


Now let's create the tags. I am using only two tags, but of course you can create as many as you need.

In [2]:
# Make two tags in the new project
Up_tag = trainer.create_tag(project.id, "Up")
Down_tag = trainer.create_tag(project.id, "Down")

Now I am reading the images from a directory - again, remember to change the directory with the one you will use - and then I am uploading the images into the Custom Vision service, adding the related tags.

In [3]:
import os

# Change the directory with the one you will be using
Up_dir = "Images\\Train\\Up"
List_of_images = []
List_of_tags = []
# Upload all the images in the folder with the same 'Up' tag
for image in os.listdir(os.fsencode(Up_dir)):
    with open(Up_dir + "\\" + os.fsdecode(image), mode="rb") as img_data: 
        List_of_images.append(ImageFileCreateEntry(name=os.path.basename(image), contents = img_data.read()))
        List_of_tags.append(Up_tag.id)
trainer.create_images_from_files(project.id, List_of_images, tag_ids = List_of_tags)

# Change the directory with the one you will be using
Down_dir = "Images\\Train\\Down"
List_of_images = []
List_of_tags = []
# Upload all the images in the folder with the same 'Down' tag
for image in os.listdir(os.fsencode(Down_dir)):
    with open(Down_dir + "\\" + os.fsdecode(image), mode="rb") as img_data: 
        List_of_images.append(ImageFileCreateEntry(name=os.path.basename(image), contents = img_data.read()))
        List_of_tags.append(Down_tag.id)
trainer.create_images_from_files(project.id, List_of_images, tag_ids = List_of_tags)

Now let's train the model. The code is exactly the same provided in the sample code documentation.

In [4]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


And finally let's predict the tags of some images. The only difference here from the sample code provided in the service is that here I am using the "no store" API, which does not store the predicted image in the service. You can change it with the standard API call if you need to store your predicted images within the Custom Vision service.

Unfortunately there is not currently an API to predict multiple images with one call.

In [5]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

# Open the sample image and get back the prediction results.
with open("Images\\test\\200000.png", mode="rb") as test_data:
     results = predictor.predict_image_with_no_store(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

	Up: 87.55%
	Down: 3.57%
